<a href="https://colab.research.google.com/github/Mr-houngbo/respire/blob/main/prediction/predict_iqa_others_location.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor

from sklearn.model_selection import GridSearchCV, TimeSeriesSplit, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import joblib

In [65]:
!pip install -q gdown
import gdown

# ID des Datasets et noms des outputs
file_ids = ["1BZazndHP417b_nUGeSxmB-wdaYDb6Ngk","1YXLAh-6muUdr_OozqJKX-NKtqQrD6Oh3","1T_k88fw5VGXLElxMLnfZZjVYW-10tJsU","14icTE7xhjxpWL_yetgMY0e0nGAlAV3Zz","1RwCELDqcmiayG-hGTf-8G2iNB9Pmsbrd"]
outputs = ["iqa-164928.csv","iqa-151726.csv","iqa-90106.csv","iqa-90104.csv","iqa-89441.csv"]


for i,file_id in enumerate(file_ids):
  url = f"https://drive.google.com/uc?id={file_id}"
  output = outputs[i]
  gdown.download(url, output, quiet=False)


Downloading...
From: https://drive.google.com/uc?id=1BZazndHP417b_nUGeSxmB-wdaYDb6Ngk
To: /content/iqa-164928.csv
100%|██████████| 427/427 [00:00<00:00, 987kB/s]
Downloading...
From: https://drive.google.com/uc?id=1YXLAh-6muUdr_OozqJKX-NKtqQrD6Oh3
To: /content/iqa-151726.csv
100%|██████████| 1.86k/1.86k [00:00<00:00, 4.44MB/s]
Downloading...
From: https://drive.google.com/uc?id=1T_k88fw5VGXLElxMLnfZZjVYW-10tJsU
To: /content/iqa-90106.csv
100%|██████████| 1.39k/1.39k [00:00<00:00, 3.43MB/s]
Downloading...
From: https://drive.google.com/uc?id=14icTE7xhjxpWL_yetgMY0e0nGAlAV3Zz
To: /content/iqa-90104.csv
100%|██████████| 1.86k/1.86k [00:00<00:00, 4.19MB/s]
Downloading...
From: https://drive.google.com/uc?id=1RwCELDqcmiayG-hGTf-8G2iNB9Pmsbrd
To: /content/iqa-89441.csv
100%|██████████| 750/750 [00:00<00:00, 2.70MB/s]


In [66]:
listt = ["164928","151726","90106","90104","89441"]

# df_164928 = pd.read_csv("iqa-164928.csv")
df_151726 = pd.read_csv("iqa-151726.csv")
df_90106 = pd.read_csv("iqa-90106.csv")
df_90104 = pd.read_csv("iqa-90104.csv")
df_89441 = pd.read_csv("iqa-89441.csv")


In [67]:
dfs = {id_: pd.read_csv(f"iqa-{id_}.csv") for id_ in listt}

##151726

In [68]:
df_151726.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    98 non-null     object 
 1   iqa     98 non-null     float64
dtypes: float64(1), object(1)
memory usage: 1.7+ KB


In [69]:

# Linear Regression — time-series friendly (lags, TSCV, eval, save, multi-step forecast)
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.base import clone
import joblib

# === 1) Données ===
df = df_151726.copy()
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date').set_index('date').asfreq('D')
df['iqa'] = df['iqa'].interpolate()


# === 2) Lags ===
def create_lags(data: pd.DataFrame, n_lags: int) -> pd.DataFrame:
    out = data.copy()
    for lag in range(1, n_lags + 1):
        out[f'lag_{lag}'] = out['iqa'].shift(lag)
    return out

n_lags = 7
df_lags = create_lags(df[['iqa']], n_lags).dropna()

# === 3) Split train/test ===
train_size = int(len(df_lags) * 0.8)
train = df_lags.iloc[:train_size].copy()
test  = df_lags.iloc[train_size:].copy()

X_train, y_train = train.drop(columns=['iqa']), train['iqa']
X_test,  y_test  = test.drop(columns=['iqa']),  test['iqa']

# === 4) CV temporelle (RMSE par fold)
n_splits = max(2, min(5, len(X_train) - 1))
tscv = TimeSeriesSplit(n_splits=n_splits)

base_lr = make_pipeline(
    StandardScaler(with_mean=True, with_std=True),
    LinearRegression(n_jobs=None)  # n_jobs not supported; kept for symmetry
)

def cv_rmse(model, X, y, splitter):
    rmses = []
    for tr_idx, val_idx in splitter.split(X):
        X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]
        est = clone(model)
        est.fit(X_tr, y_tr)
        y_hat = est.predict(X_val)
        rmses.append(np.sqrt(mean_squared_error(y_val, y_hat)))
    return np.array(rmses)

cv_rmse_per_fold = cv_rmse(base_lr, X_train, y_train, tscv)
print("📊 CV RMSE per fold :", np.round(cv_rmse_per_fold, 4))
print("📊 CV RMSE mean     :", np.round(cv_rmse_per_fold.mean(), 4))

# === 5) Entraînement final (val interne 20% — pas d’early stopping pour LR)
val_cut = max(int(len(X_train) * 0.8), 1)
X_tr, y_tr = X_train.iloc[:val_cut], y_train.iloc[:val_cut]
X_val, y_val = X_train.iloc[val_cut:], y_train.iloc[val_cut:]

final_lr = make_pipeline(
    StandardScaler(with_mean=True, with_std=True),
    LinearRegression()
)
final_lr.fit(X_tr, y_tr)

# === 6) Évaluation test ===
y_pred = final_lr.predict(X_test)
rmse_test = float(np.sqrt(mean_squared_error(y_test, y_pred)))
mae_test  = float(mean_absolute_error(y_test, y_pred))
r2_test   = float(r2_score(y_test, y_pred))

print("🔎 Évaluation sur test :")
print(f"   RMSE = {rmse_test:.4f}")
print(f"   MAE  = {mae_test:.4f}")
print(f"   R2   = {r2_test:.4f}")

# === 7) Sauvegarde ===
joblib.dump(final_lr, f"linreg_iqa_best_{151726}.pkl")
print(f"✅ Modèle sauvegardé : linreg_iqa_best_{151726}.pkl")

# === 8) Prédiction J+1 ===
last_feats = df_lags.drop(columns=['iqa']).iloc[-1:].copy()
pred_next_day = float(final_lr.predict(last_feats)[0])
print(f"📅 Prédiction J+1 : {pred_next_day:.2f}")

# === 9) Prédictions J+1 → J+5 (auto-régression) ===
n_days = 5
multi_preds = []
lag_cols = [f'lag_{k}' for k in range(1, n_lags + 1)]
step_feats = last_feats.copy()

for _ in range(n_days):
    y_hat = float(final_lr.predict(step_feats)[0])
    multi_preds.append(y_hat)
    shifted = np.roll(step_feats[lag_cols].to_numpy().ravel(), 1)
    shifted[0] = y_hat
    step_feats[lag_cols] = shifted

print("📅 Prédictions J+1 à J+5 :", [round(p, 2) for p in multi_preds])



📊 CV RMSE per fold : [6.8969 4.0716 2.3245 4.1009 2.5387]
📊 CV RMSE mean     : 3.9865
🔎 Évaluation sur test :
   RMSE = 5.4077
   MAE  = 4.0387
   R2   = -0.0789
✅ Modèle sauvegardé : linreg_iqa_best_151726.pkl
📅 Prédiction J+1 : 124.58
📅 Prédictions J+1 à J+5 : [124.58, 119.71, 121.94, 120.99, 120.59]


##90106

In [70]:
df_90106.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86 entries, 0 to 85
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    86 non-null     object 
 1   iqa     44 non-null     float64
dtypes: float64(1), object(1)
memory usage: 1.5+ KB


In [71]:
df_90106 = df_90106.dropna()

In [72]:

# Linear Regression — time-series friendly (lags, TSCV, eval, save, multi-step forecast)
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.base import clone
import joblib

# === 1) Données ===
df = df_90106.copy()
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date').set_index('date').asfreq('D')
df['iqa'] = df['iqa'].interpolate()


# === 2) Lags ===
def create_lags(data: pd.DataFrame, n_lags: int) -> pd.DataFrame:
    out = data.copy()
    for lag in range(1, n_lags + 1):
        out[f'lag_{lag}'] = out['iqa'].shift(lag)
    return out

n_lags = 7
df_lags = create_lags(df[['iqa']], n_lags).dropna()

# === 3) Split train/test ===
train_size = int(len(df_lags) * 0.8)
train = df_lags.iloc[:train_size].copy()
test  = df_lags.iloc[train_size:].copy()

X_train, y_train = train.drop(columns=['iqa']), train['iqa']
X_test,  y_test  = test.drop(columns=['iqa']),  test['iqa']

# === 4) CV temporelle (RMSE par fold)
n_splits = max(2, min(5, len(X_train) - 1))
tscv = TimeSeriesSplit(n_splits=n_splits)

base_lr = make_pipeline(
    StandardScaler(with_mean=True, with_std=True),
    LinearRegression(n_jobs=None)  # n_jobs not supported; kept for symmetry
)

def cv_rmse(model, X, y, splitter):
    rmses = []
    for tr_idx, val_idx in splitter.split(X):
        X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]
        est = clone(model)
        est.fit(X_tr, y_tr)
        y_hat = est.predict(X_val)
        rmses.append(np.sqrt(mean_squared_error(y_val, y_hat)))
    return np.array(rmses)

cv_rmse_per_fold = cv_rmse(base_lr, X_train, y_train, tscv)
print("📊 CV RMSE per fold :", np.round(cv_rmse_per_fold, 4))
print("📊 CV RMSE mean     :", np.round(cv_rmse_per_fold.mean(), 4))

# === 5) Entraînement final (val interne 20% — pas d’early stopping pour LR)
val_cut = max(int(len(X_train) * 0.8), 1)
X_tr, y_tr = X_train.iloc[:val_cut], y_train.iloc[:val_cut]
X_val, y_val = X_train.iloc[val_cut:], y_train.iloc[val_cut:]

final_lr = make_pipeline(
    StandardScaler(with_mean=True, with_std=True),
    LinearRegression()
)
final_lr.fit(X_tr, y_tr)

# === 6) Évaluation test ===
y_pred = final_lr.predict(X_test)
rmse_test = float(np.sqrt(mean_squared_error(y_test, y_pred)))
mae_test  = float(mean_absolute_error(y_test, y_pred))
r2_test   = float(r2_score(y_test, y_pred))

print("🔎 Évaluation sur test :")
print(f"   RMSE = {rmse_test:.4f}")
print(f"   MAE  = {mae_test:.4f}")
print(f"   R2   = {r2_test:.4f}")

# === 7) Sauvegarde ===
joblib.dump(final_lr, f"linreg_iqa_best_{90106}.pkl")
print(f"✅ Modèle sauvegardé : linreg_iqa_best_{90106}.pkl")

# === 8) Prédiction J+1 ===
last_feats = df_lags.drop(columns=['iqa']).iloc[-1:].copy()
pred_next_day = float(final_lr.predict(last_feats)[0])
print(f"📅 Prédiction J+1 : {pred_next_day:.2f}")

# === 9) Prédictions J+1 → J+5 (auto-régression) ===
n_days = 5
multi_preds = []
lag_cols = [f'lag_{k}' for k in range(1, n_lags + 1)]
step_feats = last_feats.copy()

for _ in range(n_days):
    y_hat = float(final_lr.predict(step_feats)[0])
    multi_preds.append(y_hat)
    shifted = np.roll(step_feats[lag_cols].to_numpy().ravel(), 1)
    shifted[0] = y_hat
    step_feats[lag_cols] = shifted

print("📅 Prédictions J+1 à J+5 :", [round(p, 2) for p in multi_preds])



📊 CV RMSE per fold : [52.2576 75.5688 49.2218 18.0473  9.8582]
📊 CV RMSE mean     : 40.9907
🔎 Évaluation sur test :
   RMSE = 28.9748
   MAE  = 15.2997
   R2   = 0.1135
✅ Modèle sauvegardé : linreg_iqa_best_90106.pkl
📅 Prédiction J+1 : 142.05
📅 Prédictions J+1 à J+5 : [142.05, 150.78, 158.77, 167.87, 176.28]


## 90104

In [73]:
df_90104.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    98 non-null     object 
 1   iqa     98 non-null     float64
dtypes: float64(1), object(1)
memory usage: 1.7+ KB


In [74]:

# Linear Regression — time-series friendly (lags, TSCV, eval, save, multi-step forecast)
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.base import clone
import joblib

# === 1) Données ===
df = df_90104.copy()
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date').set_index('date').asfreq('D')
df['iqa'] = df['iqa'].interpolate()


# === 2) Lags ===
def create_lags(data: pd.DataFrame, n_lags: int) -> pd.DataFrame:
    out = data.copy()
    for lag in range(1, n_lags + 1):
        out[f'lag_{lag}'] = out['iqa'].shift(lag)
    return out

n_lags = 7
df_lags = create_lags(df[['iqa']], n_lags).dropna()

# === 3) Split train/test ===
train_size = int(len(df_lags) * 0.8)
train = df_lags.iloc[:train_size].copy()
test  = df_lags.iloc[train_size:].copy()

X_train, y_train = train.drop(columns=['iqa']), train['iqa']
X_test,  y_test  = test.drop(columns=['iqa']),  test['iqa']

# === 4) CV temporelle (RMSE par fold)
n_splits = max(2, min(5, len(X_train) - 1))
tscv = TimeSeriesSplit(n_splits=n_splits)

base_lr = make_pipeline(
    StandardScaler(with_mean=True, with_std=True),
    LinearRegression(n_jobs=None)  # n_jobs not supported; kept for symmetry
)

def cv_rmse(model, X, y, splitter):
    rmses = []
    for tr_idx, val_idx in splitter.split(X):
        X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]
        est = clone(model)
        est.fit(X_tr, y_tr)
        y_hat = est.predict(X_val)
        rmses.append(np.sqrt(mean_squared_error(y_val, y_hat)))
    return np.array(rmses)

cv_rmse_per_fold = cv_rmse(base_lr, X_train, y_train, tscv)
print("📊 CV RMSE per fold :", np.round(cv_rmse_per_fold, 4))
print("📊 CV RMSE mean     :", np.round(cv_rmse_per_fold.mean(), 4))

# === 5) Entraînement final (val interne 20% — pas d’early stopping pour LR)
val_cut = max(int(len(X_train) * 0.8), 1)
X_tr, y_tr = X_train.iloc[:val_cut], y_train.iloc[:val_cut]
X_val, y_val = X_train.iloc[val_cut:], y_train.iloc[val_cut:]

final_lr = make_pipeline(
    StandardScaler(with_mean=True, with_std=True),
    LinearRegression()
)
final_lr.fit(X_tr, y_tr)

# === 6) Évaluation test ===
y_pred = final_lr.predict(X_test)
rmse_test = float(np.sqrt(mean_squared_error(y_test, y_pred)))
mae_test  = float(mean_absolute_error(y_test, y_pred))
r2_test   = float(r2_score(y_test, y_pred))

print("🔎 Évaluation sur test :")
print(f"   RMSE = {rmse_test:.4f}")
print(f"   MAE  = {mae_test:.4f}")
print(f"   R2   = {r2_test:.4f}")

# === 7) Sauvegarde ===
joblib.dump(final_lr, f"linreg_iqa_best_{90104}.pkl")
print(f"✅ Modèle sauvegardé : linreg_iqa_best_{90104}.pkl")

# === 8) Prédiction J+1 ===
last_feats = df_lags.drop(columns=['iqa']).iloc[-1:].copy()
pred_next_day = float(final_lr.predict(last_feats)[0])
print(f"📅 Prédiction J+1 : {pred_next_day:.2f}")

# === 9) Prédictions J+1 → J+5 (auto-régression) ===
n_days = 5
multi_preds = []
lag_cols = [f'lag_{k}' for k in range(1, n_lags + 1)]
step_feats = last_feats.copy()

for _ in range(n_days):
    y_hat = float(final_lr.predict(step_feats)[0])
    multi_preds.append(y_hat)
    shifted = np.roll(step_feats[lag_cols].to_numpy().ravel(), 1)
    shifted[0] = y_hat
    step_feats[lag_cols] = shifted

print("📅 Prédictions J+1 à J+5 :", [round(p, 2) for p in multi_preds])



📊 CV RMSE per fold : [11.5149  6.2823  3.3327  4.8162  4.9307]
📊 CV RMSE mean     : 6.1754
🔎 Évaluation sur test :
   RMSE = 4.7308
   MAE  = 3.9633
   R2   = 0.4102
✅ Modèle sauvegardé : linreg_iqa_best_90104.pkl
📅 Prédiction J+1 : 128.13
📅 Prédictions J+1 à J+5 : [128.13, 134.02, 134.34, 132.5, 129.27]


## 89441

In [75]:
df_89441.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    39 non-null     object 
 1   iqa     39 non-null     float64
dtypes: float64(1), object(1)
memory usage: 756.0+ bytes


In [76]:

# Linear Regression — time-series friendly (lags, TSCV, eval, save, multi-step forecast)
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.base import clone
import joblib

# === 1) Données ===
df = df_89441.copy()
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date').set_index('date').asfreq('D')
df['iqa'] = df['iqa'].interpolate()


# === 2) Lags ===
def create_lags(data: pd.DataFrame, n_lags: int) -> pd.DataFrame:
    out = data.copy()
    for lag in range(1, n_lags + 1):
        out[f'lag_{lag}'] = out['iqa'].shift(lag)
    return out

n_lags = 7
df_lags = create_lags(df[['iqa']], n_lags).dropna()

# === 3) Split train/test ===
train_size = int(len(df_lags) * 0.8)
train = df_lags.iloc[:train_size].copy()
test  = df_lags.iloc[train_size:].copy()

X_train, y_train = train.drop(columns=['iqa']), train['iqa']
X_test,  y_test  = test.drop(columns=['iqa']),  test['iqa']

# === 4) CV temporelle (RMSE par fold)
n_splits = max(2, min(5, len(X_train) - 1))
tscv = TimeSeriesSplit(n_splits=n_splits)

base_lr = make_pipeline(
    StandardScaler(with_mean=True, with_std=True),
    LinearRegression(n_jobs=None)  # n_jobs not supported; kept for symmetry
)

def cv_rmse(model, X, y, splitter):
    rmses = []
    for tr_idx, val_idx in splitter.split(X):
        X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]
        est = clone(model)
        est.fit(X_tr, y_tr)
        y_hat = est.predict(X_val)
        rmses.append(np.sqrt(mean_squared_error(y_val, y_hat)))
    return np.array(rmses)

cv_rmse_per_fold = cv_rmse(base_lr, X_train, y_train, tscv)
print("📊 CV RMSE per fold :", np.round(cv_rmse_per_fold, 4))
print("📊 CV RMSE mean     :", np.round(cv_rmse_per_fold.mean(), 4))

# === 5) Entraînement final (val interne 20% — pas d’early stopping pour LR)
val_cut = max(int(len(X_train) * 0.8), 1)
X_tr, y_tr = X_train.iloc[:val_cut], y_train.iloc[:val_cut]
X_val, y_val = X_train.iloc[val_cut:], y_train.iloc[val_cut:]

final_lr = make_pipeline(
    StandardScaler(with_mean=True, with_std=True),
    LinearRegression()
)
final_lr.fit(X_tr, y_tr)

# === 6) Évaluation test ===
y_pred = final_lr.predict(X_test)
rmse_test = float(np.sqrt(mean_squared_error(y_test, y_pred)))
mae_test  = float(mean_absolute_error(y_test, y_pred))
r2_test   = float(r2_score(y_test, y_pred))

print("🔎 Évaluation sur test :")
print(f"   RMSE = {rmse_test:.4f}")
print(f"   MAE  = {mae_test:.4f}")
print(f"   R2   = {r2_test:.4f}")

# === 7) Sauvegarde ===
joblib.dump(final_lr, f"linreg_iqa_best_{89441}.pkl")
print(f"✅ Modèle sauvegardé : linreg_iqa_best_{89441}.pkl")

# === 8) Prédiction J+1 ===
last_feats = df_lags.drop(columns=['iqa']).iloc[-1:].copy()
pred_next_day = float(final_lr.predict(last_feats)[0])
print(f"📅 Prédiction J+1 : {pred_next_day:.2f}")

# === 9) Prédictions J+1 → J+5 (auto-régression) ===
n_days = 5
multi_preds = []
lag_cols = [f'lag_{k}' for k in range(1, n_lags + 1)]
step_feats = last_feats.copy()

for _ in range(n_days):
    y_hat = float(final_lr.predict(step_feats)[0])
    multi_preds.append(y_hat)
    shifted = np.roll(step_feats[lag_cols].to_numpy().ravel(), 1)
    shifted[0] = y_hat
    step_feats[lag_cols] = shifted

print("📅 Prédictions J+1 à J+5 :", [round(p, 2) for p in multi_preds])



📊 CV RMSE per fold : [ 6.0292  3.8879 10.2312  6.2934  4.8017]
📊 CV RMSE mean     : 6.2487
🔎 Évaluation sur test :
   RMSE = 6.1670
   MAE  = 4.2326
   R2   = -0.2270
✅ Modèle sauvegardé : linreg_iqa_best_89441.pkl
📅 Prédiction J+1 : 114.38
📅 Prédictions J+1 à J+5 : [114.38, 118.39, 120.25, 123.09, 120.46]


In [77]:
"""# CECI EST UN TEST D'UTILISATION DU MODELE XGB


# XGBoost — time-series friendly (lags, TSCV, eval with early stopping, save, multi-step forecast)
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.base import clone
import joblib

# === 1) Données ===
df = df_151726.copy()
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date').set_index('date').asfreq('D')
df['iqa'] = df['iqa'].interpolate()

# === 2) Lags ===
def create_lags(data: pd.DataFrame, n_lags: int) -> pd.DataFrame:
    out = data.copy()
    for lag in range(1, n_lags + 1):
        out[f'lag_{lag}'] = out['iqa'].shift(lag)
    return out

n_lags = 7
df_lags = create_lags(df[['iqa']], n_lags).dropna()

# === 3) Split train/test ===
train_size = int(len(df_lags) * 0.8)
train = df_lags.iloc[:train_size].copy()
test  = df_lags.iloc[train_size:].copy()

X_train, y_train = train.drop(columns=['iqa']), train['iqa']
X_test,  y_test  = test.drop(columns=['iqa']),  test['iqa']

# === 4) CV temporelle (RMSE par fold) — avec early stopping
n_splits = max(2, min(5, len(X_train) - 1))
tscv = TimeSeriesSplit(n_splits=n_splits)

base_xgb = XGBRegressor(
    n_estimators=2000,
    learning_rate=0.03,
    max_depth=3,
    subsample=0.9,
    colsample_bytree=0.9,
    reg_alpha=0.1,
    reg_lambda=1.0,
    random_state=42,
    n_jobs=-1,
    tree_method="hist",
    objective="reg:squarederror",
    eval_metric="rmse",
)

def cv_rmse(model, X, y, splitter):
    rmses = []
    for tr_idx, val_idx in splitter.split(X):
        X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]
        est = clone(model)
        est.fit(
            X_tr, y_tr,
            eval_set=[(X_val, y_val)],
            verbose=False
        )
        best_iter = getattr(est, "best_iteration", None)
        if best_iter is not None:
            try:
                y_hat = est.predict(X_val, iteration_range=(0, best_iter + 1))
            except TypeError:
                # Compat old xgboost
                y_hat = est.predict(X_val, ntree_limit=getattr(est, "best_ntree_limit", 0))
        else:
            y_hat = est.predict(X_val)
        rmses.append(np.sqrt(mean_squared_error(y_val, y_hat)))
    return np.array(rmses)

cv_rmse_per_fold = cv_rmse(base_xgb, X_train, y_train, tscv)
print("📊 CV RMSE per fold :", np.round(cv_rmse_per_fold, 4))
print("📊 CV RMSE mean     :", np.round(cv_rmse_per_fold.mean(), 4))

# === 5) Entraînement final + early stopping (val interne 20%)
val_cut = max(int(len(X_train) * 0.8), 1)
X_tr, y_tr = X_train.iloc[:val_cut], y_train.iloc[:val_cut]
X_val, y_val = X_train.iloc[val_cut:], y_train.iloc[val_cut:]

final_xgb = clone(base_xgb)
final_xgb.fit(
    X_tr, y_tr,
    eval_set=[(X_val, y_val)],
    verbose=False
)
best_iter = getattr(final_xgb, "best_iteration", None)
print("🏁 Best iteration (ES):", best_iter)

# === 6) Évaluation test ===
if best_iter is not None:
    try:
        y_pred = final_xgb.predict(X_test, iteration_range=(0, best_iter + 1))
    except TypeError:
        y_pred = final_xgb.predict(X_test, ntree_limit=getattr(final_xgb, "best_ntree_limit", 0))
else:
    y_pred = final_xgb.predict(X_test)

rmse_test = float(np.sqrt(mean_squared_error(y_test, y_pred)))
mae_test  = float(mean_absolute_error(y_test, y_pred))
r2_test   = float(r2_score(y_test, y_pred))

print("🔎 Évaluation sur test :")
print(f"   RMSE = {rmse_test:.4f}")
print(f"   MAE  = {mae_test:.4f}")
print(f"   R2   = {r2_test:.4f}")

# === 7) Sauvegarde ===
joblib.dump(final_xgb, f"xgb_iqa_best_{151726}.pkl")
print(f"✅ Modèle sauvegardé : xgb_iqa_best_{151726}.pkl")

# === 8) Prédiction J+1 ===
last_feats = df_lags.drop(columns=['iqa']).iloc[-1:].copy()
if best_iter is not None:
    try:
        pred_next_day = float(final_xgb.predict(last_feats, iteration_range=(0, best_iter + 1))[0])
    except TypeError:
        pred_next_day = float(final_xgb.predict(last_feats, ntree_limit=getattr(final_xgb, "best_ntree_limit", 0))[0])
else:
    pred_next_day = float(final_xgb.predict(last_feats)[0])
print(f"📅 Prédiction J+1 : {pred_next_day:.2f}")

# === 9) Prédictions J+1 → J+5 (auto-régression) ===
n_days = 5
multi_preds = []
lag_cols = [f'lag_{k}' for k in range(1, n_lags + 1)]
step_feats = last_feats.copy()

for _ in range(n_days):
    if best_iter is not None:
        try:
            y_hat = float(final_xgb.predict(step_feats, iteration_range=(0, best_iter + 1))[0])
        except TypeError:
            y_hat = float(final_xgb.predict(step_feats, ntree_limit=getattr(final_xgb, "best_ntree_limit", 0))[0])
    else:
        y_hat = float(final_xgb.predict(step_feats)[0])
    multi_preds.append(y_hat)
    shifted = np.roll(step_feats[lag_cols].to_numpy().ravel(), 1)
    shifted[0] = y_hat
    step_feats[lag_cols] = shifted

print("📅 Prédictions J+1 à J+5 :", [round(p, 2) for p in multi_preds])
"""

'# CECI EST UN TEST D\'UTILISATION DU MODELE XGB\n\n\n# XGBoost — time-series friendly (lags, TSCV, eval with early stopping, save, multi-step forecast)\nimport pandas as pd\nimport numpy as np\nfrom xgboost import XGBRegressor\nfrom sklearn.model_selection import TimeSeriesSplit\nfrom sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score\nfrom sklearn.base import clone\nimport joblib\n\n# === 1) Données ===\ndf = df_151726.copy()\ndf[\'date\'] = pd.to_datetime(df[\'date\'])\ndf = df.sort_values(\'date\').set_index(\'date\').asfreq(\'D\')\ndf[\'iqa\'] = df[\'iqa\'].interpolate()\n\n# === 2) Lags ===\ndef create_lags(data: pd.DataFrame, n_lags: int) -> pd.DataFrame:\n    out = data.copy()\n    for lag in range(1, n_lags + 1):\n        out[f\'lag_{lag}\'] = out[\'iqa\'].shift(lag)\n    return out\n\nn_lags = 7\ndf_lags = create_lags(df[[\'iqa\']], n_lags).dropna()\n\n# === 3) Split train/test ===\ntrain_size = int(len(df_lags) * 0.8)\ntrain = df_lags.iloc[:train_size].